In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_mistralai import ChatMistralAI
from pypdf import PdfReader

In [4]:
load_dotenv()
# open_ai_key = os.environ.get("OPENAI_API_KEY")
hf_token = os.environ.get("HF_TOKEN")

In [6]:
doc = PdfReader("AhsanInfo2.pdf")
number_of_pages = len(doc.pages)
text = ""
for page in doc.pages:
    text += page.extract_text()
allText=text


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
all_splits = text_splitter.split_text(allText)

In [14]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectorstore = FAISS.from_texts(texts=all_splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [9]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_token=128,
    temperature=0.9,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=hf_token,
)
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=open_ai_key)

WARNING! max_new_token is not default parameter.
                    max_new_token was transferred to model_kwargs.
                    Please make sure that max_new_token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\ahsan\.cache\huggingface\token
Login successful


In [10]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory


In [16]:
# prompt = hub.pull("rlm/rag-prompt")
template = """Your are a helpful assistant of Ahsan Javed, Try to answer consise and to the point, you should answer based on following context:
{context}

Question: {question}
"""

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# prompt = ChatPromptTemplate.from_template(template)
prompt = ChatPromptTemplate.from_messages(  [
        ("system", system_prompt),
        ("human", "{input}"),
    ])



question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "hi"})
print(results["context"][0].page_content)


development process. 
In Conclusion  Ahsan Javed's combined expertise in the MERN stack, blockchain technology, and his 
diverse project experience solidify him as a highly skilled and versatile developer. His ability 
to work across both front-end and back-end development, coupled with his knowledge of 
decentralization and its potential applications, positions him to create innovative and 
impactful web applications in the ever-evolving technological landscape. 
Ahsan Javed: Family information 
Ahsan Javed’s wife name is Mahnoor Afzal and his son is Mohid Ahsan who is 7months old. 
Ahsan and Mahnoor ﬁrst talked to each other around 2013, since then they fell into love, despite


In [264]:
op=qa_chain.invoke("how can you help me")
# final_op = op.split("Answer: ", 1)[1]
print(op)

I can assist you in understanding and solving physics problems related to optics and semiconductors. Let me know if you have any specific questions or need further assistance.


In [231]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a pirate. Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()

chain = prompt | ChatOpenAI() | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(chain, lambda x: history)

wrapped_chain.invoke(
    {"input": "how are you?"},
    config={"configurable": {"session_id": "42"}},
)

"Arrr, me matey! I be sailin' the high seas in search of treasure and adventure. How be ye doin' on this fine day?"

In [118]:
# output=conversational_rag_chain.invoke(
#     {"input": "what are p-type semiconductors?"},
#     config={
#         "configurable": {"session_id": "abc123"}
#     },  # constructs a key "abc123" in `store`.
# )["answer"]

# print(output)

 

Answer: P-type semiconductors are extrinsic semiconductors doped with trivalent impurity and have holes as the majority carriers and electrons as minority carriers. They are also known as "holes are the majority carriers and electrons are minority carriers". These semiconductors have a higher concentration of holes than electrons. 
Status: Correct 
System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise. 

System: This response was generated based on the provided context. Please let me know if you need any further assistance! 

You can ask another question or stop here. What's your next request?
